In [ ]:
!kaggle datasets download -d mateuszbuda/lgg-mri-segmentation

In [ ]:
!unzip lgg-mri-segmentation.zip

In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [8]:
train_images=[]
train_masked_images=[]

for patinet_files in  os.listdir('/content/kaggle_3m'):
  if patinet_files.startswith('TCGA'):
    for images in  os.listdir('/content/kaggle_3m/'+patinet_files):
      if images.endswith('_mask.tif'):
        train_masked_images.append('/content/kaggle_3m/'+patinet_files+'/'+images)
        train_images.append('/content/kaggle_3m/'+patinet_files+'/'+images.replace('_mask',''))


In [9]:
from PIL import Image
from sklearn.model_selection import train_test_split

def pre_images(image_path):
    image = Image.open(image_path)
    image = image.resize((256, 256))
    image = np.array(image)
    image = image / 255.0
    return image

pre_train_masked_images = []
pre_train_images = []

for image_path in train_masked_images[:500]:
    pre_train_masked_images.append(pre_images(image_path))

for image_path in train_images[:500]:
    pre_train_images.append(pre_images(image_path))

pre_train_masked_images = np.array(pre_train_masked_images)
pre_train_images = np.array(pre_train_images)

train_images, test_images, train_masked_images, test_masked_images = train_test_split(pre_train_images, pre_train_masked_images, test_size=0.2, random_state=42)


In [10]:
pre_train_images=np.expand_dims(train_images,axis=-1)
pre_train_masked_images=np.expand_dims(train_masked_images,axis=-1)

pre_test_images=np.expand_dims(test_images,axis=-1)
pre_test_masked_images=np.expand_dims(test_masked_images,axis=-1)

In [ ]:
pre_test_images[0]

In [ ]:
# make sure i have  only 1 and zero in the masked image
import numpy as np
unique_values = np.unique(pre_train_masked_images)
print(f"Unique values in pre_train_masked_images: {unique_values}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

num_images = 4
random_indices = np.random.randint(0, len(pre_train_images), size=num_images)

fig, axes = plt.subplots(2, 4, figsize=(15, 8))

for idx, i in enumerate(random_indices):
    axes[0, idx].imshow(pre_train_images[i].squeeze())
    axes[0, idx].set_title(f" image ")
    axes[0, idx].axis('off')

    axes[1, idx].imshow(pre_train_masked_images[i].squeeze(), cmap="gray")
    axes[1, idx].set_title(f"Mask")
    axes[1, idx].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import tensorflow as tf

inputs = tf.keras.layers.Input(shape=(256, 256, 3))

conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='conv1_1')(inputs)
conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='conv1_2')(conv1)
pool1 = tf.keras.layers.MaxPooling2D((2, 2), name='pool1')(conv1)

conv2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv2_1')(pool1)
conv2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv2_2')(conv2)
pool2 = tf.keras.layers.MaxPooling2D((2, 2), name='pool2')(conv2)

conv3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='conv3_1')(pool2)
conv3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='conv3_2')(conv3)
pool3 = tf.keras.layers.MaxPooling2D((2, 2), name='pool3')(conv3)

conv4 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='conv4_1')(pool3)
conv4 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='conv4_2')(conv4)
pool4 = tf.keras.layers.MaxPooling2D((2, 2), name='pool4')(conv4)

conv5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_1')(pool4)
conv5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_2')(conv5)

up6 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same', name='up6')(conv5)
merge6 = tf.keras.layers.concatenate([up6, conv4], name='merge6')
conv6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='conv6_1')(merge6)
conv6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='conv6_2')(conv6)

up7 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same', name='up7')(conv6)
merge7 = tf.keras.layers.concatenate([up7, conv3], name='merge7')
conv7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='conv7_1')(merge7)
conv7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='conv7_2')(conv7)

up8 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same', name='up8')(conv7)
merge8 = tf.keras.layers.concatenate([up8, conv2], name='merge8')
conv8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv8_1')(merge8)
conv8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv8_2')(conv8)

up9 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same', name='up9')(conv8)
merge9 = tf.keras.layers.concatenate([up9, conv1], name='merge9')
conv9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='conv9_1')(merge9)
conv9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='conv9_2')(conv9)

outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid', name='outputs')(conv9)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()


In [ ]:
initial_learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)

def scheduler(epoch, lr):
    if epoch % 10 == 0 and epoch != 0:
        return lr * 0.1
    else:
        return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(pre_train_images, pre_train_masked_images, epochs=30,validation_data=[test_images,test_masked_images], batch_size=1, callbacks=[lr_scheduler])

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('/content/drive/MyDrive/MyModel_Segmentation_MRI_2.h5')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for i in range(100):
    test_image = pre_train_images[i]
    test_mask = pre_train_masked_images[i]
    predicted_mask = model.predict(np.expand_dims(test_image, axis=0))[0]

    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(np.squeeze(test_image))
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(test_mask)
    plt.title('Ground Truth Mask')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(predicted_mask)
    plt.title('Predicted Mask')
    plt.axis('off')

    plt.show()
